# 💡 MosaicX: **Example: Simple pendulum**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**隐函数最优显式化 - 机器科学家应用**

In this notebook, we will walk through the theoretical and computational foundation of **dimensional analysis**, with a focus on the **Buckingham Pi theorem**. You will learn:

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: DDDA Project, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*


# 1. 单摆控制方程

下面用**能量守恒**把单摆方程一步步推出来；这是最直接、最“由守恒律来”的推导。结论会是

$$
\boxed{\ \ddot\theta+\frac{g}{L}\sin\theta=0\ }
$$

**理想化假设**：细而不可伸长的轻绳/刚杆，摆球质量 $m$，摆长 $L$，铰接无摩擦，仅受重力，无空气阻力。

---

## 1.1 普适化处理
在普适化处理阶段，我们仅采用**无量纲化**以消除单位影响并凸显方程的本征结构。由于研究重点在于**函数的结构分析**及**状态分布**的保持，诸如归一化、标准化与正则化等数值预处理方法均不适用，因此在此环节中被排除。

* **无量纲化**：令 $\tau = t\sqrt{g/L}$，则

  $$
  \theta''(\tau) + \sin\theta = 0,
  $$

  其中 $'$ 为对 $\tau$ 的导数。

---

## 1.2 结构化转化
在进入 Jacobian 分析之前，我们首先需要对单摆方程进行结构化转化，将原始的二阶非线性ODE改写为更适合分析的形式（如一阶系统或残差形式），以便在后续能够从不同视角开展稳定性、可逆性与函数结构的判定。

### 1.2.1 一阶系统化（state-space form）

$$
x_1 = \theta, \quad x_2 = \theta' \,,
$$

系统变为：

$$
\begin{cases}
x_1' = x_2 \\
x_2' = -\sin(x_1)
\end{cases}
$$

这个形式便于写成向量函数：

$$
\mathbf{x}' = f(\mathbf{x}) = 
\begin{bmatrix}
x_2 \\
-\sin(x_1)
\end{bmatrix}
$$

---

### 1.2.2 残差形式（residual form）

$$
F(\theta, \theta', \theta'') = \theta'' + \sin\theta = 0
$$

* 在这个视角下，$(\theta, \theta', \theta'')$ 是独立变量；
* 方程 $F=0$ 定义了一个“函数流形”；
* 你可以对 $F$ 做偏导，得到残差形式下的 Jacobian。

---

## 1.3 Jacobian 的两种切入点

* **一阶系统视角**：Jacobian 是

  $$
  J = \frac{\partial f}{\partial x} 
  = \begin{bmatrix}
  \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} \\
  \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}
  \end{bmatrix}
  =
  \begin{bmatrix}
  0 & 1 \\
  -\cos(x_1) & 0
  \end{bmatrix}
  $$

* **残差形式视角**：Jacobian 是

  $$
  J = \left[ \frac{\partial F}{\partial \theta}, \frac{\partial F}{\partial \theta'}, \frac{\partial F}{\partial \theta''} \right]
  = \big[ \cos\theta, \; 0, \; 1 \big]
  $$

前者用于稳定性分析（动力系统的流形局部线性化），后者用于隐函数分析（解的显函数化、函数结构判定）。

---



# 2. 单摆模型函数结构解析
我们要对单摆模型的Jacobian使用

> 在 MosaicX 的框架下，我们通常从残差形式出发，以保证对动力系统与非动力系统的一致处理。然而，在 Jacobian 的数值诊断（如行列式、条件数和奇异值分解）上，残差形式往往退化为非方阵，不利于谱特性的定量化分析。因此在数值层面，我们进一步采用一阶系统化的转化，以获得方阵 Jacobian，从而更稳定地开展 det、cond 与 $\sigma_{\min}$ 的计算。
---

## 2.1 Jacobian格式选择

残差形式天然的能够揭示**函数流形的存在性与显函数化条件**，但在数值诊断上往往因 Jacobian 退化为非方阵而受限，无法直接开展行列式、条件数或奇异值分析。相比之下，一阶系统化在动力系统场景下提供了一个方阵 Jacobian，使得谱特性分析得以顺利进行，从而更适合 det/cond/SVD 等定量化诊断。

需要强调的是：

* **适用范围**：这一结论主要成立于动力系统（ODE/PDE）类问题中，因为它们天然可以转化为状态空间形式；
* **无损性**：从残差形式到一阶系统形式，本质上是对高阶导数引入中间变量的“重写”，方程的信息并未丢失，因此结果是无损的；
* **非动力系统**：对于一般非动力学约束问题（纯代数方程或守恒关系），我们对模型会有封闭性要求，所以方阵问题不存在。

### 2.1.1残差形式的局限

在残差形式下，比如单摆

$$
F(\theta, \theta', \theta'') = \theta'' + \sin\theta = 0,
$$

Jacobian 是一行向量：

$$
J = \big[\cos\theta,\;0,\;1\big].
$$

* 这是一个 **1×3 矩阵**，没有方阵结构：

  * $\det(J)$ 没有定义；
  * $\kappa(J)$（条件数）不具备传统意义；
  * SVD 的最小奇异值 $\sigma_{\min}$ 也只给出“这个行向量的最小拉伸”，无法反映整体结构的可逆性。
* 在单摆的例子中，**残差形式适合逻辑判定（能否显函数化），但不适合数值诊断（det/cond/SVD）**。

---

### 2.1.2 一阶系统化的优势

把它转为一阶系统：

$$
\begin{cases}
x_1' = x_2 \\
x_2' = -\sin(x_1)
\end{cases}
$$

对应向量场

$$
f(x_1,x_2) = \begin{bmatrix} x_2 \\ -\sin(x_1)\end{bmatrix},
$$

Jacobian 是

$$
J = \begin{bmatrix}
0 & 1 \\
-\cos(x_1) & 0
\end{bmatrix}.
$$

* 这是一个 **方阵 (2×2)**：

  * $\det(J) = \cos(x_1)$ 可用于奇异性判定；
  * $\kappa(J)$ 条件数反映了局部线性化的敏感性；
  * SVD 的奇异值给出主方向上的放缩因子，能量化系统的局部刚性/脆弱性。

所以在这个例子中 **一阶系统视角更适合做数值诊断 (det/cond/SVD)**。

---


In [1]:
import sympy as sp

# --- 1) 无量纲单摆：theta'' + sin(theta) = 0 ---
tau = sp.symbols('tau')           # 无量纲时间
theta = sp.Function('theta')(tau)

# --- 2) 一阶系统化 ---
x1, x2 = sp.symbols('x1 x2')      # x1 = theta, x2 = theta'
f1 = x2
f2 = -sp.sin(x1)

f = sp.Matrix([f1, f2])
x = sp.Matrix([x1, x2])

# --- 3) Jacobian ---
J = f.jacobian(x)                  # ∂f/∂x
sp.pprint(J)

# 可选：给出数值点处的评估（例如 x1 = pi/3, x2 = 0.2）
J_num = J.subs({x1: sp.pi/3, x2: 0.2})
print("\nJ(pi/3, 0.2) =")
sp.pprint(J_num)


⎡   0      1⎤
⎢           ⎥
⎣-cos(x₁)  0⎦

J(pi/3, 0.2) =
⎡ 0    1⎤
⎢       ⎥
⎣-1/2  0⎦


In [ ]:
## 2.2 

对的！拿到 **一阶系统的 Jacobian**

$$
J(\theta)=\begin{bmatrix}0&1\\-\cos\theta&0\end{bmatrix}
$$

之后，就可以把你离散数据里的每个 $\theta_i$ **直接代入**来算指标了。

**保守单摆的闭式结果（超省事）：**

* 行列式：$\det J_i=\cos\theta_i$
* 奇异值：$\sigma_{\min,i}=|\cos\theta_i|$、$\sigma_{\max,i}=1$
* 条件数：$\kappa_i=1/|\cos\theta_i|$

**数值例子（弧度）：**

* $\theta=0$: $\det=1$, $\sigma_{\min}=1$, $\kappa=1$（最稳）
* $\theta=\pi/3\approx1.047$: $\cos\theta=0.5\Rightarrow \det=0.5$, $\sigma_{\min}=0.5$, $\kappa=2$
* $\theta=\pi/2-0.01$: $\cos\theta\approx0.01\Rightarrow \sigma_{\min}\approx0.01$, $\kappa\approx100$（接近奇异）
* $\theta=\pi/2$: $\cos\theta=0\Rightarrow \sigma_{\min}=0$, $\kappa=\infty$（奇异边界）

**实操清单：**

1. 确保 $\theta$ 用弧度（必要时 `unwrap`）。
2. 对每个样本计算 $|\cos\theta_i|$；设阈值 $\tau$（如 0.05），做分区：

   * 良态：$|\cos\theta_i|\ge\tau$
   * 边界：$|\cos\theta_i|<\tau$
3. 画时间序列或相图热力图（$|\cos\theta|$、$\kappa$）。
4. 在良态区做你后续的显函数拟合；靠近 $\pi/2+k\pi$ 的样本谨慎/分片处理。

> 若日后加入阻尼 $\zeta$：$J=\begin{bmatrix}0&1\\-\cos\theta&-2\zeta\end{bmatrix}$。这时用样本点数值算 SVD/特征值即可（$\det=\cos\theta$ 仍然是奇异判据）。
